In [1]:
!pip install tensorflow-hub --no-deps
!pip install tensorflow-datasets --no-deps


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
import os
import numpy as np

import tensorflow as tf
#import tensorflow_hub as hub
#import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
#print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.14.0
Eager mode:  True
GPU is available


2024-06-20 11:53:37.092601: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 11:53:37.126280: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 11:53:37.126320: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [5]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

2024-06-18 17:03:20.516401: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [3]:
import tf_keras as ktf

In [6]:
model = ktf.Sequential()
model.add(hub_layer)
model.add(ktf.layers.Dense(16, activation='relu'))
model.add(ktf.layers.Dense(1))

In [7]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48191433 (183.84 MB)
Trainable params: 48191433 (183.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [9]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = ktf.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(train_data.shuffle(10000).batch(512),
           epochs=10,
            validation_data=validation_data.batch(512),
          verbose=1)

Epoch 1/10
30/30 [==============================] - 25s 454ms/step - loss: 0.6608 - accuracy: 0.5412 - val_loss: 0.5998 - val_accuracy: 0.6205
Epoch 2/10
30/30 [==============================] - 13s 445ms/step - loss: 0.5264 - accuracy: 0.7065 - val_loss: 0.4840 - val_accuracy: 0.7593
Epoch 3/10
30/30 [==============================] - 13s 442ms/step - loss: 0.3906 - accuracy: 0.8280 - val_loss: 0.3942 - val_accuracy: 0.8297
Epoch 4/10
30/30 [==============================] - 13s 442ms/step - loss: 0.2841 - accuracy: 0.8917 - val_loss: 0.3428 - val_accuracy: 0.8541
Epoch 5/10
30/30 [==============================] - 13s 449ms/step - loss: 0.2108 - accuracy: 0.9253 - val_loss: 0.3196 - val_accuracy: 0.8663
Epoch 6/10
30/30 [==============================] - 13s 443ms/step - loss: 0.1570 - accuracy: 0.9513 - val_loss: 0.3067 - val_accuracy: 0.8651
Epoch 7/10
30/30 [==============================] - 13s 445ms/step - loss: 0.1162 - accuracy: 0.9665 - val_loss: 0.3036 - val_accuracy: 0.8703

In [19]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

print(history, log_dir)
writer = tf.summary.create_file_writer(log_dir)

with writer.as_default():
    for epoch, (train_loss, val_loss, acc, val_acc) in enumerate(zip(history.history['loss'], history.history['val_loss'], history.history['accuracy'], history.history['val_accuracy'])):
        print(epoch, train_loss, val_loss)
        tf.summary.scalar('loss', train_loss, step=epoch)
        tf.summary.scalar('val_loss', val_loss, step=epoch)
        tf.summary.scalar('accuracy', acc, step=epoch)
        tf.summary.scalar('val_acc', val_acc, step=epoch)

        writer.flush()
        # Ajouter d'autres métriques si nécessaire

<tf_keras.src.callbacks.History object at 0x7fd62825d350> logs/fit/20240618-184959
0 0.6607714295387268 0.5998238325119019
1 0.5264492034912109 0.48396065831184387
2 0.39057764410972595 0.3942417502403259
3 0.28414714336395264 0.34279686212539673
4 0.21081192791461945 0.3195594549179077
5 0.1569909155368805 0.3066900670528412
6 0.11622925847768784 0.30360376834869385
7 0.08534103631973267 0.3083646297454834
8 0.06241045147180557 0.3144962787628174
9 0.045731306076049805 0.325151264667511


In [18]:
history.history

{'loss': [0.6607714295387268,
  0.5264492034912109,
  0.39057764410972595,
  0.28414714336395264,
  0.21081192791461945,
  0.1569909155368805,
  0.11622925847768784,
  0.08534103631973267,
  0.06241045147180557,
  0.045731306076049805],
 'accuracy': [0.5411999821662903,
  0.7065333127975464,
  0.828000009059906,
  0.8916666507720947,
  0.925266683101654,
  0.9513333439826965,
  0.9664666652679443,
  0.9787333607673645,
  0.98826664686203,
  0.9933333396911621],
 'val_loss': [0.5998238325119019,
  0.48396065831184387,
  0.3942417502403259,
  0.34279686212539673,
  0.3195594549179077,
  0.3066900670528412,
  0.30360376834869385,
  0.3083646297454834,
  0.3144962787628174,
  0.325151264667511],
 'val_accuracy': [0.6205000281333923,
  0.7592999935150146,
  0.8296999931335449,
  0.8540999889373779,
  0.8662999868392944,
  0.8651000261306763,
  0.8702999949455261,
  0.8697999715805054,
  0.8743000030517578,
  0.8727999925613403]}